In [1]:
import pandas as pd
import numpy as np
from transformers import BertTokenizer, TFBertForSequenceClassification
from transformers import InputExample, InputFeatures

import tensorflow as tf

In [2]:
model = TFBertForSequenceClassification.from_pretrained("bert-base-uncased")
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")

All model checkpoint layers were used when initializing TFBertForSequenceClassification.

Some layers of TFBertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [3]:
model.summary()

Model: "tf_bert_for_sequence_classification"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
bert (TFBertMainLayer)       multiple                  109482240 
_________________________________________________________________
dropout_37 (Dropout)         multiple                  0         
_________________________________________________________________
classifier (Dense)           multiple                  1538      
Total params: 109,483,778
Trainable params: 109,483,778
Non-trainable params: 0
_________________________________________________________________


In [4]:
df = pd.read_csv('clean_data_for _BERT.csv')

In [5]:
pd.set_option('display.max_colwidth', None)

In [6]:
df.head()

,tweet,neg,neu,pos,compound,tidy_tweet,label
0,should i doordash myself a bagel?,0.000,1.000,0.000,0.0000,should doordash myself bagel,0.0
1,"In the spirit of social distancing... 🍕🍜🍔🍟🥘🍱🍧🥧🌮🥪🍓 DoorDash is a great way to support local restaurants with safe, no-contact delivery. Get $15 off ($5 off each of your first 3 orders) when you sign up with this link: https://t.co/XITN1b5FKB",0.000,0.820,0.180,0.8860,spirit social distanc doordash great support local restaur with safe contact deliveri each your first order when sign with thi link http xitn,1.0
2,Recent math: $23M from @Pinterest to @FrancoiseBr / $2.5M anti- discrimination org donation / $0 @IfeomaOzoma @erikashimizu $3B ea. @DoorDash founders / $200M Main Street Strong program $6B ea. @Airbnb founders / $238M in shares for hosts / ~$50M to .org 🤔 these ratios,0.046,0.843,0.110,0.4939,recent math from anti discrimin donat founder main street strong program founder share host these ratio,1.0
3,@DoorDash I want McDonald’s,0.000,0.698,0.302,0.0772,want mcdonald,1.0
4,"You should try DoorDash, it's an awesome food delivery service. Get $7.00 off your next order when you sign up with this link: https://t.co/u3ITGwbuT8.",0.000,0.849,0.151,0.6249,should doordash awesom food deliveri servic your next order when sign with thi link http itgwbut,1.0


In [7]:
df = df.drop(['neg', 'neu', 'pos', 'compound', 'tidy_tweet'], axis=1)
df.reset_index(drop=True)

,tweet,label
0,should i doordash myself a bagel?,0.0
1,"In the spirit of social distancing... 🍕🍜🍔🍟🥘🍱🍧🥧🌮🥪🍓 DoorDash is a great way to support local restaurants with safe, no-contact delivery. Get $15 off ($5 off each of your first 3 orders) when you sign up with this link: https://t.co/XITN1b5FKB",1.0
2,Recent math: $23M from @Pinterest to @FrancoiseBr / $2.5M anti- discrimination org donation / $0 @IfeomaOzoma @erikashimizu $3B ea. @DoorDash founders / $200M Main Street Strong program $6B ea. @Airbnb founders / $238M in shares for hosts / ~$50M to .org 🤔 these ratios,1.0
3,@DoorDash I want McDonald’s,1.0
4,"You should try DoorDash, it's an awesome food delivery service. Get $7.00 off your next order when you sign up with this link: https://t.co/u3ITGwbuT8.",1.0
...,...,...
395,@DoorDash_Help worst customer service - 140 line que to tell you my food was not delivered? Ya... using this again.:.,-1.0
396,@_cjv1 don’t you got a car ?? or doordash??,0.0
397,"@couchtomato62 @SiakamRunsFast @CaseyJLynn @malika_andrews @MarcJSpears Agreed - same punishment for all. Only other guy to get same punishment, walked out hotel lobby door to pickup Doordash. Then came back in. He got a 10 day. Is this scale of justice the same? One was far riskier than the other. At least games hadn't started 1st time. No $ risk.",-1.0
398,who’s tryna ft and order door dash and have lunch tg,0.0


In [8]:
train = df.iloc[:320, :]
test = df.iloc[320:, :]

In [9]:
train.shape

(320, 2)

In [10]:
test.shape

(80, 2)

In [11]:
def convert_data_to_examples(train, test, DATA_COLUMN, LABEL_COLUMN): 
  train_InputExamples = train.apply(lambda x: InputExample(guid=None, # Globally unique ID for bookkeeping, unused in this case
                                                          text_a = x[DATA_COLUMN], 
                                                          text_b = None,
                                                          label = x[LABEL_COLUMN]), axis = 1)

  validation_InputExamples = test.apply(lambda x: InputExample(guid=None, # Globally unique ID for bookkeeping, unused in this case
                                                          text_a = x[DATA_COLUMN], 
                                                          text_b = None,
                                                          label = x[LABEL_COLUMN]), axis = 1)
  
  return train_InputExamples, validation_InputExamples

  train_InputExamples, validation_InputExamples = convert_data_to_examples(train, 
                                                                           test, 
                                                                           'DATA_COLUMN', 
                                                                           'LABEL_COLUMN')
  
def convert_examples_to_tf_dataset(examples, tokenizer, max_length=128):
    features = [] # -> will hold InputFeatures to be converted later

    for e in examples:
        # Documentation is really strong for this method, so please take a look at it
        input_dict = tokenizer.encode_plus(
            e.text_a,
            add_special_tokens=True,
            max_length=max_length, # truncates if len(s) > max_length
            return_token_type_ids=True,
            return_attention_mask=True,
            #pad_to_max_length=True, # pads to the right by default # CHECK THIS for pad_to_max_length
            truncation=True
        )

        input_ids, token_type_ids, attention_mask = (input_dict["input_ids"],
            input_dict["token_type_ids"], input_dict['attention_mask'])

        features.append(
            InputFeatures(
                input_ids=input_ids, attention_mask=attention_mask, token_type_ids=token_type_ids, label=e.label
            )
        )

    def gen():
        for f in features:
            yield (
                {
                    "input_ids": f.input_ids,
                    "attention_mask": f.attention_mask,
                    "token_type_ids": f.token_type_ids,
                },
                f.label,
            )

    return tf.data.Dataset.from_generator(
        gen,
        ({"input_ids": tf.int32, "attention_mask": tf.int32, "token_type_ids": tf.int32}, tf.int64),
        (
            {
                "input_ids": tf.TensorShape([None]),
                "attention_mask": tf.TensorShape([None]),
                "token_type_ids": tf.TensorShape([None]),
            },
            tf.TensorShape([]),
        ),
    )


DATA_COLUMN = 'tweet'
LABEL_COLUMN = 'label'

In [12]:
train_InputExamples, validation_InputExamples = convert_data_to_examples(train, test, DATA_COLUMN, LABEL_COLUMN)

train_data = convert_examples_to_tf_dataset(list(train_InputExamples), tokenizer)
train_data = train_data.shuffle(100).batch(32).repeat(2)

validation_data = convert_examples_to_tf_dataset(list(validation_InputExamples), tokenizer)
validation_data = validation_data.batch(32)

In [ ]:
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=3e-5, epsilon=1e-08, clipnorm=1.0), 
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True), 
              metrics=[tf.keras.metrics.SparseCategoricalAccuracy('accuracy')])

model.fit(train_data, epochs=2, validation_data=validation_data)

Epoch 1/2
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module, class, method, function, traceback, frame, or code object was expected, got cython_function_or_method
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module, class, method, function, traceback, frame, or code object was expected, got cython_function_or_method
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
